<p style="font-size:20px">Using Simulated Annealing to solve an assignment problem</p>
<p style="font-size:16px">The goal is to assign 7 clients to specific depots/locations at minimum cost such that only 3 depots are utilized. There is a cost associated with each specific client/depot combination. </p>

<p style="font-size:20px">Import libraries</p>

In [29]:
import random
import math
from copy import deepcopy

<p style="font-size:20px">Goal Function</p>
<p style="font-size:16px">This function evaluates the total cost of the client assignment/configuration. 
If the number of depots utilized is greater than 3, a penalty is imposed at a rate of alpha for each
depot over 3.</p>

In [30]:
def goal(Cost, clients):
    total = 0
    alpha = 100
    for i,j in clients:
        total += Cost[j][i]
        
    # PENALTY FOR MORE THAN 3 DEPOTS
    open_depots = []
    for i,j in clients:
        open_depots.append(j)
    depot_penalty = (len(set(open_depots)))
    if depot_penalty > 3:
        total += alpha*(depot_penalty-3)
    return (total)

<p style="font-size:20px">Define function to assign clients to a random depot</p>

In [31]:
def assign_client(clients):
    # Generate a random number between 0 and 1. Use this number to pick a client
    # and randomly assign it to a new depot
    clients1 = deepcopy(clients)
    rand = random.random()

    other_depots = [0,1,2,3,4,5]
    if rand >= 0 and rand < (1/7):
        #assign client 1 to a new, random depot
        del other_depots[clients1[0][1]]
        clients1[0][1] = random.choice(other_depots)
    if rand >= (1/7) and rand < (2/7):
        #assign client 2 to a new, random depot
        del other_depots[clients1[1][1]]
        clients1[1][1] = random.choice(other_depots)
    if rand >= (2/7) and rand < (3/7):
        #assign client 3 to a new, random depot
        del other_depots[clients1[2][1]]
        clients1[2][1] = random.choice(other_depots)
    if rand >= (3/7) and rand < (4/7):
        #assign client 4 to a new, random depot
        del other_depots[clients1[3][1]]
        clients1[3][1] = random.choice(other_depots)
    if rand >= (4/7) and rand < (5/7):
        #assign client 5 to a new, random depot
        del other_depots[clients1[4][1]]
        clients1[4][1] = random.choice(other_depots)
    if rand >= (5/7) and rand < (6/7):
        #assign client 6 to a new, random depot
        del other_depots[clients1[5][1]]
        clients1[5][1] = random.choice(other_depots)
    if rand >= (6/7):
        #assign client 7 to a new, random depot
        del other_depots[clients1[6][1]]
        clients1[6][1] = random.choice(other_depots)

    return (clients1)

<p style="font-size:20px">Define cost matrix and initial client/depot assignments</p>

In [38]:
# Transposed Cost matrix:
Cost = {0:[2,3,6,8,4,2,6],
        1:[3,1,2,1,4,8,5],
        2:[7,1,3,4,3,3,3],
        3:[3,8,1,6,3,6,2],
        4:[6,10,2,2,4,3,7],
        5:[1,4,7,3,3,2,4]}

# Clients are represented as client number and then the depot they are assigned to
# Client 0 is actually client 1 and depot 1 actually represents depot 2.

clients = [[0,0],
           [1,1],
           [2,2],
           [3,3],
           [4,4],
           [5,5],
           [6,1]]

<p style="font-size:20px">Simulated Annealing:</p>
<p style="font-size:16px">Begin with random assignments for clients/depots (done above). Make a random change to the assignment and evaluate the goal function. The new assignment replaces the incumbant solution with probability 1 if the new one has a better goal value, and with some probability between 0 and 1 if it has a worse goal value. </p>
<p style="font-size:16px">The probability of accepting a worse solution is proportional to the difference in goal values, so slightly worse solutions have a high probability of being accepted, while much worse solutions will only be accepted infrequently. Therefore if the number of iterations is sufficiently large, it means that one can move away from any local minimum. However, for the process to converge in the long run, the probability of accepting worse solutions decreases over time, so the algorithm should end up converging at a "good" local minimum.</p>
<p style="font-size:16px">Note that delta represents the change in the goal value. The larger delta is, the less chance there is of making a move to a solution worse by delta. Also, as the temperature (T) decreases, the chances of making a move to a worse solution decrease.</p>

In [39]:
T = 10
r = 0.90
L = 2
best_solution = (['placeholder'],1000)
# while not yet frozen
print ("Initial Client Assignments: {0}".format(clients))
print ("Initial Objective: {0}".format(goal(Cost, clients)))
print ("----------------------------------------------------------------------------")
while T > .01:
    while L > 0:
        clients_prime = assign_client(clients)
        delta = goal(Cost, clients_prime) - goal(Cost, clients)
        if delta <= 0:
            print ("----------------------------------------------------------------------------")
            print ("New assignment gives better goal function value. Its delta is: {0}".format(delta))
            print ("Old depot assignments: {0}".format(clients))
            print ("New depot assignments: {0}".format(clients_prime))
            clients = clients_prime
            print ("Objective: {0}".format(goal(Cost, clients)))
            if goal(Cost, clients) < best_solution[1]:
                best_solution = (clients, goal(Cost, clients))
            
        elif random.random() <= math.e**((-1*delta)/T):
            print ("***************************PROBABILITY UPDATE*******************************")
            print ("Choosing a worse solution with a probability of {0}".format(math.e**((-1*delta)/T)))
            print ("Old depot assignments: {0}".format(clients))
            print ("New depot assignments: {0}".format(clients_prime))
            clients = clients_prime
            print ("Objective: {0}".format(goal(Cost, clients)))
            
        L = L - 1
        #print (L)
    L = 2
    #print (T)
    T = T*r
print ("The best solution found had an objective of {0}\
       \nand assignments of {1}".format(best_solution[1], best_solution[0]))

Initial Client Assignments: [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 1]]
Initial Objective: 323
----------------------------------------------------------------------------
----------------------------------------------------------------------------
New assignment gives better goal function value. Its delta is: -96
Old depot assignments: [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 1]]
New depot assignments: [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 3], [6, 1]]
Objective: 227
----------------------------------------------------------------------------
New assignment gives better goal function value. Its delta is: -101
Old depot assignments: [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 3], [6, 1]]
New depot assignments: [[0, 0], [1, 1], [2, 2], [3, 3], [4, 3], [5, 3], [6, 1]]
Objective: 126
----------------------------------------------------------------------------
New assignment gives better goal function value. Its delta is: -4
Old depot assignments: [[0, 0], [